# Test Notebook
Load a trained model to show the performance.

In [2]:
from model.phinet_v3 import PhiNetV3
import torch
import torch.nn as nn
from micromind import PhiNet

import torchvision
import torchvision.transforms as transforms

# Avalanche modules
from avalanche.benchmarks.classic import SplitCIFAR10

import utility.utils as utils
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

c:\Users\matte\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def test(dataset, model, criterion, device, eval_mb_size):       
        sum_accuracy = 0
        exps_acc = dict()
        results = [[],[]]
        for exp in dataset:
            print("Testing task ", exp.task_label)
            print('Classes in this task:', exp.classes_in_this_experience)

            experience_dataloader = DataLoader(exp.dataset, batch_size=eval_mb_size, shuffle=False)
            test_acc, test_loss = utils.test(model, criterion, experience_dataloader, device)
            sum_accuracy += test_acc
            print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")
            exps_acc[exp.task_label] = test_acc

            results[0].append(f"Task {exp.task_label}")
            results[1].append(test_acc)

        # Calculate and add average accuracy
        avg_accuracy = sum_accuracy / len(dataset)
        results[0].append(f"Avg Acc")
        results[1].append(avg_accuracy)

        print(f"Average accuracy: {avg_accuracy:.2f}%")

# Define a function to visualize predictions for a specific task
def visualize_predictions_for_task(model, dataset, device, num_images=5):
    data_loader = DataLoader(
            dataset,
            batch_size=16,
            shuffle=True,
        )
    classes = dataset.targets
    classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    count = 0
    with torch.no_grad():
        for batch in data_loader:
            inputs, targets, _ = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            for i in range(len(targets)):
                image = np.transpose(inputs[i].cpu().numpy(), (1, 2, 0))
                true_label = classes[targets[i]]

                predicted_label = classes[predicted[i]]

                plt.imshow(image)
                plt.title(f'True: {true_label}, Predicted: {predicted_label}')
                plt.show()

                count += 1
                if count >= num_images:
                    return


In [ ]:
transform = transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224, 224)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

split_cifar = SplitCIFAR10(n_experiences=5, seed=0, return_task_id = True, train_transform = transform, eval_transform = transform)

# recovering the train and test streams
train_stream = split_cifar.train_stream
test_stream = split_cifar.test_stream

torch.manual_seed(0)
# Loss criterion
criterion = nn.CrossEntropyLoss()

input_shape = (3, 224, 224)
torch.cuda.set_device(2) if torch.cuda.is_available() else None

# Set the device as cuda, the GPU specified as default will be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(f"Device: {device}")    

phinet = PhiNet(input_shape = input_shape, alpha = 3, beta = 0.75, t_zero = 6, num_layers=7 ,include_top = False, num_classes = 1000).to(device)

model = PhiNetV3(phinet, latent_layer_num = 10).to(device)

# Load the saved state dictionary
saved_state_dict = torch.load("./results/latent_10/weight_decay_0/lr_5e-05_epochs_10_rm_MB_325_rm_None_split_0.8.pth", map_location=torch.device(device))  # Replace with the actual path

# Load the state dictionary into the model
model.load_state_dict(saved_state_dict)

test(test_stream, model, criterion, device, eval_mb_size=16)

model.eval()  # Set the model to evaluation mode

for exp in test_stream:
    visualize_predictions_for_task(model, exp.dataset, device, num_images=5)

In [4]:
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(f"Device: {device}")  

model = PhiNet.from_pretrained("ImageNet-1k",2.0, 0.75, 6.0, 7, 224, num_classes=1000, path="phinet2.pth.tar", classifier=False, device = device)

print(model)
#summary(model, input_size=(1, 3, 224, 224))


Device: cpu
Checkpoint taken from local file system.
Checkpoint loaded successfully.
PhiNet(
  (_layers): ModuleList(
    (0): ZeroPad2d((0, 1, 0, 1))
    (1): SeparableConv2d(
      (_layers): ModuleList(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), groups=3, bias=False)
        (1): Conv2d(3, 110, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(110, eps=0.001, momentum=0.999, affine=True, track_running_stats=True)
        (3): Hardswish()
      )
    )
    (2): PhiNetConvBlock(
      (_layers): ModuleList(
        (0): Dropout2d(p=0.05, inplace=False)
        (1): DepthwiseConv2d(110, 110, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=110, bias=False)
        (2): BatchNorm2d(110, eps=0.001, momentum=0.999, affine=True, track_running_stats=True)
        (3): Hardswish()
        (4): Conv2d(110, 55, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (5): BatchNorm2d(55, eps=0.001, momentum=0.999, affine=True, track_running_stats=

In [4]:
model2 = PhiNetV3(model, latent_layer_num = 10).to(device)

summary(model2, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
PhiNetV3                                      [1, 10]                   --
├─Sequential: 1-1                             [1, 441, 7, 7]            --
│    └─ZeroPad2d: 2-1                         [1, 3, 225, 225]          --
│    └─SeparableConv2d: 2-2                   [1, 110, 112, 112]        --
│    │    └─ModuleList: 3-1                   --                        577
│    └─PhiNetConvBlock: 2-3                   [1, 55, 112, 112]         --
│    │    └─ModuleList: 3-2                   --                        7,370
│    └─PhiNetConvBlock: 2-4                   [1, 55, 56, 56]           --
│    │    └─ModuleList: 3-3                   --                        56,025
│    └─PhiNetConvBlock: 2-5                   [1, 55, 56, 56]           --
│    │    └─ModuleList: 3-4                   --                        52,895
│    └─PhiNetConvBlock: 2-6                   [1, 110, 28, 28]          --
│    │  

In [27]:
import torch

x = torch.rand(5,3, dtype=torch.float32)

print(f"x: {x}")   
print(f"x.dtype: {x.dtype}")
print(f"Dim in B: {x.element_size() * x.nelement()}")


x = torch.quantize_per_tensor(x, 1.0, 0, torch.quint8)

print(f"x: {x}")   
print(f"x.dtype: {x.dtype}")
print(f"Dim in B: {x.element_size() * x.nelement()}")

x = torch.dequantize(x)

print(f"x: {x}")
print(f"x.dtype: {x.dtype}")
print(f"Dim in B: {x.element_size() * x.nelement()}")


x: tensor([[0.5145, 0.0246, 0.7773],
        [0.2114, 0.8546, 0.9567],
        [0.1866, 0.3325, 0.4517],
        [0.7924, 0.1082, 0.0326],
        [0.6336, 0.4259, 0.4195]])
x.dtype: torch.float32
Dim in B: 60
x: tensor([[1., 0., 1.],
        [0., 1., 1.],
        [0., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]], size=(5, 3), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=1.0, zero_point=0)
x.dtype: torch.quint8
Dim in B: 15
x: tensor([[1., 0., 1.],
        [0., 1., 1.],
        [0., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
x.dtype: torch.float32
Dim in B: 60
